# OCR com Rede Neural

O problema de reconhecer caracteres expressos em imagens é conhecido como
Optical Character Recognition (OCR).

Mostraremos aqui como treinar uma Rede Neural em Python para
reconhecer dígitos, para ser aplicada no nosso problema
de quebrar CAPTCHAS vulneráveis!

## Dependências

In [0]:
# Processamento de dados
import pandas as pd
# Processamento numérico
import numpy as np
# Visualização
import matplotlib.pyplot as plt
# Aprendizado de máquina
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
# Processamento de imagens
import cv2

## Coleta e engenharia de *features*

Para treinar a rede neural, precisamos de **exemplos**. No nosso
caso, são imagens de dígitos.

Vamos assumir aqui que temos um banco de dados
com várias imagens contendo dígitos, e veremos
como extraí-los e prepará-los para o treino da rede,
usando uma imagem de exemplo.

Primeiro, ganharemos acesso à imagem, armazenada no drive:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Definindo o caminho dos arquivos para o nosso projeto.
root_path = "drive/My Drive/campus-party-captchas"

Agora, vamos carregar essa imagem, e exibi-la para vermos o resultado:

In [0]:
# Criando o dataset
example = cv2.imread(root_path + '/dataset/example.png', 0)

plt.imshow(example, cmap='gray')
plt.show

Vamos agora limpar essa imagem, e binarizá-la, para isolar os dígitos:

In [0]:
binary = cv2.adaptiveThreshold(example,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,75,10)
plt.figure()
plt.imshow(binary, cmap='gray')
plt.show

binary = cv2.medianBlur(binary, 3)
binary = cv2.medianBlur(binary, 3)
binary = cv2.medianBlur(binary, 3)
binary = cv2.medianBlur(binary, 3)

plt.figure()
plt.imshow(binary, cmap='gray')
plt.show


Agora, repetimos o mesmo processamento para
segmentar os dígitos, desde a busca de contornos, 
ordenação e correção de inclinação.

Ao final, produzimos os vetores de características
que representam os dígitos.

In [0]:
SZ_W = 15
SZ_H = 20

In [0]:
_, contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Obtendo os contornos

# Definição da função que retorna o primeiro ponto a esquerda do contorno
def top_left(c):
    x,y,_,_ = cv2.boundingRect(c)
    return (x,y)

contours.sort(key=len,reverse=True) # Realizando a ordenação pelo tamanho (de maneira decrescente)
contours = contours[:min(6, len(contours))] # Obtendo os seis primeiros contornos
contours.sort(key=top_left) # Sorteando os 6 (ou menos) contornos pela posição

# Definindo a função que corrige a orientação dos dígitos
def deskew(img):
    m = cv2.moments(img)    
    if abs(m['mu02']) < 1e-2:
        return img.copy()    
    skew = m['mu11']/m['mu02']    
    M = np.float32([[1, skew, -0.5*SZ_W*skew], [0, 1, 0]])    
    img = cv2.warpAffine(img, M, (SZ_W, SZ_H), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_CUBIC)
    return img

fig_chars = plt.figure()
number_images = []

# Itera sobre os contornos e aplica a correção
for c in range(len(contours)):
    x,y,w,h = cv2.boundingRect(contours[c])
    crop = np.zeros(binary.shape, dtype=np.uint8)
    cv2.drawContours(crop, contours, c, 255, -1)
    crop = cv2.bitwise_and(crop, binary)
    crop = crop[y:y+h, x:x+w]

    crop = cv2.resize(crop, (SZ_W, SZ_H), interpolation = cv2.INTER_CUBIC) # Redimensionando o contorno para (15x20)
    skewed = deskew(crop) # Aplicação da correção
    fig_chars.add_subplot(1, 6, c+1)
    plt.imshow(skewed, cmap = 'gray')
    number_images.append( skewed.flatten().tolist() ) # Adiciona a lista de números os dígitos selecionados.

Vamos ver com mais detalhes, para o primeiro dígito:

In [0]:
# Mostrando os pixels do primeiro dígito
print( np.array(number_images).shape )

print( np.array(number_images)[0,:])

Agora, preparamos o dataset, incluindo a resposta (target) esperada para cada vetor:

In [0]:
df = pd.DataFrame( np.array(number_images) )
df['target'] = [5,4,1,3,3,0]
df.head()

## Treinamento da Rede Neural

Vimos como geramos os vetores de características
para um digito presente em uma imagem. 

Primeiramente, carregamos o datase contendo todos os dígitos
que coletamos e transformamos:

In [0]:
dataset_path = root_path + '/dataset/dataset.csv'
dataset = pd.read_csv(dataset_path, index_col=0)
dataset.head()

Vamos explorar rapidamente esse dataset, verificando quantos exemplos de cada dígito nós temos:

In [0]:
# Obtendo informações sobre o dataset
dataset.target.value_counts()

Criaremos agora conjuntos de treino e teste. Com o primeiro, treinaremos a rede neural, e, com o segundo, avaliaremos a capacidade dela de reconhecer os dígitos:

In [0]:
# Separando os dados
train_data, test_data, train_target, test_target = train_test_split(dataset.iloc[:,:-1], dataset['target'], test_size=0.2)

In [0]:
print('[INFO] train: ', len(train_data))
print(train_target.value_counts() / len(train_target))
print('[INFO] test: ', len(test_data))
print(test_target.value_counts() / len(test_target))

Finalmente, vamos criar a nossa Rede Neural e treiná-la utilizando o dataset de treino.

In [0]:
mlp = MLPClassifier(max_iter=1000)
mlp.fit(train_data, train_target)

Por fim, avaliaremos o quão bom está o modelo, usando o dataset de teste e algumas métricas:

In [0]:
output = mlp.predict(test_data)

print('[INFO] scores')
print('[accuracy score]:', accuracy_score(np.array(test_target), output))